# Implementazione di un Sistema Esperto sugli Animali

Un esempio tratto da [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

In questo esempio, implementeremo un semplice sistema basato sulla conoscenza per determinare un animale in base ad alcune caratteristiche fisiche. Il sistema può essere rappresentato dal seguente albero AND-OR (questa è solo una parte dell'intero albero, possiamo facilmente aggiungere altre regole):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.it.png)


## Il nostro shell di sistemi esperti con inferenza retrograda

Proviamo a definire un linguaggio semplice per la rappresentazione della conoscenza basato su regole di produzione. Utilizzeremo classi Python come parole chiave per definire le regole. Ci saranno essenzialmente 3 tipi di classi:
* `Ask` rappresenta una domanda che deve essere posta all'utente. Contiene il set di risposte possibili.
* `If` rappresenta una regola ed è solo una sintassi semplificata per memorizzare il contenuto della regola.
* `AND`/`OR` sono classi per rappresentare i rami AND/OR dell'albero. Memorizzano semplicemente la lista degli argomenti al loro interno. Per semplificare il codice, tutta la funzionalità è definita nella classe genitore `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Nel nostro sistema, la memoria di lavoro conterrebbe l'elenco di **fatti** come **coppie attributo-valore**. La base di conoscenza può essere definita come un grande dizionario che associa azioni (nuovi fatti che dovrebbero essere inseriti nella memoria di lavoro) a condizioni, espresse come espressioni AND-OR. Inoltre, alcuni fatti possono essere `Ask`-ati.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Per eseguire l'inferenza inversa, definiremo la classe `Knowledgebase`. Essa conterrà:
* `Memoria` di lavoro - un dizionario che associa attributi a valori
* `Regole` della Knowledgebase nel formato definito sopra

I due metodi principali sono:
* `get` per ottenere il valore di un attributo, eseguendo l'inferenza se necessario. Ad esempio, `get('color')` recupererà il valore di uno slot colore (chiederà se necessario e memorizzerà il valore per un utilizzo successivo nella memoria di lavoro). Se chiediamo `get('color:blue')`, chiederà un colore e poi restituirà un valore `y`/`n` a seconda del colore.
* `eval` esegue l'inferenza vera e propria, ovvero attraversa l'albero AND/OR, valuta sotto-obiettivi, ecc.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Ora definiamo la nostra base di conoscenza sugli animali e svolgiamo la consultazione. Nota che questa chiamata ti farà delle domande. Puoi rispondere digitando `s`/`n` per le domande sì-no, oppure specificando un numero (0..N) per le domande con risposte a scelta multipla più lunghe.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Utilizzo di PyKnow per l'Inferenza in Avanti

Nel prossimo esempio, cercheremo di implementare l'inferenza in avanti utilizzando una delle librerie per la rappresentazione della conoscenza, [PyKnow](https://github.com/buguroo/pyknow/). **PyKnow** è una libreria per creare sistemi di inferenza in avanti in Python, progettata per essere simile al classico vecchio sistema [CLIPS](http://www.clipsrules.net/index.html).

Avremmo potuto anche implementare il chaining in avanti da soli senza troppi problemi, ma le implementazioni ingenue di solito non sono molto efficienti. Per un confronto delle regole più efficace, viene utilizzato un algoritmo speciale chiamato [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Definiremo il nostro sistema come una classe che sottoclassa `KnowledgeEngine`. Ogni regola è definita da una funzione separata con l'annotazione `@Rule`, che specifica quando la regola deve essere attivata. All'interno della regola, possiamo aggiungere nuovi fatti utilizzando la funzione `declare`, e l'aggiunta di questi fatti comporterà l'attivazione di altre regole da parte del motore di inferenza in avanti.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Una volta definito un knowledgebase, popoliamo la nostra memoria di lavoro con alcuni fatti iniziali e poi chiamiamo il metodo `run()` per eseguire l'inferenza. Come risultato, puoi vedere che nuovi fatti dedotti vengono aggiunti alla memoria di lavoro, incluso il fatto finale sull'animale (se abbiamo impostato correttamente tutti i fatti iniziali).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
